# Scraping www.huizenzoeker.nl/woningmarkt/ for all municipalities in The Netherlands

### ODCM project - Team 3 

Which municipalities in the Netherlands are hit hardest by the Dutch Housing crisis, and which the least? 
We use the site www.huizenzoeker.nl/woningmarkt/ to analyse the Dutch Housing Market, including the gem. vraagprijs, # verkochte woningen, gem. vierkante meter prijs, % overboden. 

## Step 1: Loading all the basics

In [1]:
#import the packages (after you have installed them properly)
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd 
import time 
from functools import reduce
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
# pip install webdriver-manager
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys

In [2]:
#set the basis for BeautifulSoup 
url = 'https://www.huizenzoeker.nl/woningmarkt/'
res = requests.get(url)
soup = BeautifulSoup(res.text, 'html.parser')

In [3]:
#set the basis for Selenium
chrome_path = "C:\Documents\MSc_Marketing_Analytics\oDCM\oDCM-project-team-3\src\collection\chromedriver.exe" #depends on your own path
# driver = webdriver.Chrome() 
# driver = webdriver.Chrome()(ChromeDriverManager().install()) #FEEDBACK UPDATE

## Step 2: Collecting the URLs

### Step 2a: Extracting first the provinces, and then all the municipalities 

Here we collect the URLs for all municipalities of a province using Selenium to make the process more efficient.

We first construct a base url and a province_url which once appended together creates the URL to each of the woningmarkt pages for each province. We generate the generate_links() function to append these parts of the URL together. 

In [4]:
base_url = 'https://www.huizenzoeker.nl/woningmarkt/'

In [5]:
province_url = ['noord-holland/', 'zuid-holland/', 'zeeland/', 'noord-brabant/', 'utrecht/', 'flevoland/', 
                'friesland/', 'groningen/', 'drenthe/', 'overijssel/', 'gelderland/', 'limburg/']

In [6]:
def generate_links(base_url,province_url): 
    page_links = []
    for i in province_url:
        full_links = base_url + i
        page_links.append(full_links)  
    return page_links

In [7]:
page_links = generate_links(base_url,province_url)
print(page_links)

['https://www.huizenzoeker.nl/woningmarkt/noord-holland/', 'https://www.huizenzoeker.nl/woningmarkt/zuid-holland/', 'https://www.huizenzoeker.nl/woningmarkt/zeeland/', 'https://www.huizenzoeker.nl/woningmarkt/noord-brabant/', 'https://www.huizenzoeker.nl/woningmarkt/utrecht/', 'https://www.huizenzoeker.nl/woningmarkt/flevoland/', 'https://www.huizenzoeker.nl/woningmarkt/friesland/', 'https://www.huizenzoeker.nl/woningmarkt/groningen/', 'https://www.huizenzoeker.nl/woningmarkt/drenthe/', 'https://www.huizenzoeker.nl/woningmarkt/overijssel/', 'https://www.huizenzoeker.nl/woningmarkt/gelderland/', 'https://www.huizenzoeker.nl/woningmarkt/limburg/']


We then use this list of all provinces, to extract all municipalities from each, making use of window handling (**STATUS: this code doesn't fully work yet!!**)

In [8]:
driver = webdriver.Chrome()

In [9]:
for x in range(len(page_links)): #WARNING: running this code snippet will open 12 Chrome tabs automatically!!
    driver.get(page_links[x])
    if x < 11:
        driver.execute_script("window.open('');")
        driver.switch_to.window(driver.window_handles[x+1])

In [10]:
driver.window_handles

['CDwindow-CC669956D4A1E658792951450BA22188',
 'CDwindow-D5600D8AD34333A37BAC1EB509E706C5',
 'CDwindow-60375886CA3E9BD5DD260053ED8CFCBC',
 'CDwindow-1D1E827EA866A5D2FA668F0F627FCD72',
 'CDwindow-F369D1526A4E65D2E0E7267E3275C5C1',
 'CDwindow-3B37E6FF3E658D41A36B0038CCB65704',
 'CDwindow-CEBAA1F84E669A0EAC80CF6ADB0E45BD',
 'CDwindow-6636034D3056B256F1EEC8D39F213278',
 'CDwindow-2F3CC3ED96FD588292CE624DFA2AB47E',
 'CDwindow-16419C7AA1C7C977848925BEA8A55AAA',
 'CDwindow-2697384D9C6714B39A25BD8993C14B12',
 'CDwindow-5A7B3B46F97174405390588802231698']

In [11]:
page_urls_full = []
for handle in driver.window_handles:        
    driver.switch_to.window(handle)
    elem1 = driver.find_elements_by_xpath("//li//div//a[@href]")
    
    for elem in elem1:
        urls = elem.get_attribute('href')
        page_urls_full.append(urls)   

In [12]:
page_urls_full

['https://www.huizenzoeker.nl/woningmarkt/noord-holland/aalsmeer/',
 'https://www.huizenzoeker.nl/woningmarkt/noord-holland/alkmaar/',
 'https://www.huizenzoeker.nl/woningmarkt/noord-holland/amstelveen/',
 'https://www.huizenzoeker.nl/woningmarkt/noord-holland/amsterdam/',
 'https://www.huizenzoeker.nl/woningmarkt/noord-holland/beemster/',
 'https://www.huizenzoeker.nl/woningmarkt/noord-holland/bergen-nh/',
 'https://www.huizenzoeker.nl/woningmarkt/noord-holland/beverwijk/',
 'https://www.huizenzoeker.nl/woningmarkt/noord-holland/blaricum/',
 'https://www.huizenzoeker.nl/woningmarkt/noord-holland/bloemendaal/',
 'https://www.huizenzoeker.nl/woningmarkt/noord-holland/castricum/',
 'https://www.huizenzoeker.nl/woningmarkt/noord-holland/den-helder/',
 'https://www.huizenzoeker.nl/woningmarkt/noord-holland/diemen/',
 'https://www.huizenzoeker.nl/woningmarkt/noord-holland/drechterland/',
 'https://www.huizenzoeker.nl/woningmarkt/noord-holland/edam-volendam/',
 'https://www.huizenzoeker.nl/w

In [13]:
subset = page_urls_full[0:5] # defined subset to try out on few urls first
subset

['https://www.huizenzoeker.nl/woningmarkt/noord-holland/aalsmeer/',
 'https://www.huizenzoeker.nl/woningmarkt/noord-holland/alkmaar/',
 'https://www.huizenzoeker.nl/woningmarkt/noord-holland/amstelveen/',
 'https://www.huizenzoeker.nl/woningmarkt/noord-holland/amsterdam/',
 'https://www.huizenzoeker.nl/woningmarkt/noord-holland/beemster/']

## Step 3a: Gather all data for all municipalities: Trend data

For each municipality we now extract the gem. vraagprijs, verkochte woningen, gem.vierkantemeter prijs, % overboden (and how these numbers how changed t.o.v. vorige maand).

## Gemiddelde vraagprijs (Quadrant 1 Trend data)

Extract the gemiddelde vraagprijs, and its % change t.o.v. vorige maand. 

In [14]:
#combination of beautiful.soup and selenium DOESN'T WORK!
def extract_city_trends(subset):
    trend_list = []
    for page_url in subset:
        driver.get(page_url) #FEEDBACK update
        time.sleep(5) #FEEDBACK update
        soup = BeautifulSoup(res.text, 'html.parser')
        city_name = soup.find_all('h2')[0].get_text()
        city_name1 = city_name.replace('Woningmarkt','')
        trends = soup.find_all(class_='trend-graph')[0].get_text()
        new_trend = trends.replace('\n','')
        new_trend1 = new_trend.replace('Gem. Vraagprijs','')
        price,change = new_trend1.split('                ',1)
        change1 = change.replace(' t.o.v. vorige maand            ', '')
        trend_list.append({'City':city_name1, 'Gem. vraagprijs':price, '%Δ Vraagprijs (t.o.v vorige maand)':change1})
    return(trend_list)

In [15]:
gem_vraagprijs = extract_city_trends(subset)

IndexError: list index out of range

### Try out using selenium only

In [16]:
# didnt work yet

We generate a Pandas dataframe of the data we just collected for all municipalities.

In [17]:
pd.DataFrame(gem_vraagprijs)

NameError: name 'gem_vraagprijs' is not defined

### # Verkochte woningen (Quadrant 2 Trend data)

Extract the # of verkochte woningen, and its % change t.o.v. vorige maand.

In [128]:
def extract_city_trends1(page_urls): #STATUS: change the input still to the correct list name for all municipalities of all provinces!! 
    trend_list1 = []
    for page_url in page_urls:
        driver.get(page_url) #FEEDBACK update
        time.sleep(5) #FEEDBACK update
        soup = BeautifulSoup(res.text, 'html.parser')
        city_name = soup.find_all('h2')[0].get_text()
        city_name1 = city_name.replace('Woningmarkt','')
        trends = soup.find_all(class_='trend-graph')[1].get_text()
        new_trend = trends.replace('\n','')
        new_trend1 = new_trend.replace('Verkochte woningen','')
        verkocht,change = new_trend1.split('                ',1)
        change1 = change.replace(' t.o.v. vorige maand            ', '')
        trend_list1.append({'City':city_name1, 'Verkochte woningen':verkocht, '%Δ Verkocht (t.o.v vorige maand)':change1})
    return(trend_list1)

In [129]:
verk_woningen = extract_city_trends1(page_urls) #use the function 
print(verk_woningen) #check if it works

We generate a Pandas dataframe of the data we just collected for all municipalities.

In [130]:
pd.DataFrame(verk_woningen)

,City,Verkochte woningen,%Δ Verkocht (t.o.v vorige maand)
0,Alphen-Chaam,2,-75.00%
1,Altena,20,-54.55%
2,Asten,7,-46.15%
3,Baarle-Nassau,2,-71.43%
4,Bergeijk,9,-55.00%
...,...,...,...
56,Vught,16,-42.86%
57,Waalre,4,-55.56%
58,Waalwijk,15,-65.91%
59,Woensdrecht,10,-47.37%


### Gemiddelde vierkantemeter prijs (Quadrant 3 Trend data)

Extract the gemiddelde vierkante meter prijs, and its % change t.o.v. vorige maand.

In [135]:
def extract_city_trends2(page_urls):#STATUS: change the input still to the correct list name for all municipalities of all provinces!! 
    trend_list2 = []
    for page_url in page_urls:
        driver.get(page_url) #FEEDBACK UPDATE
        time.sleep(5) #FEEDBACK UPDATE
        soup = BeautifulSoup(res.text, 'html.parser')
        city_name = soup.find_all('h2')[0].get_text()
        city_name1 = city_name.replace('Woningmarkt','')
        trends = soup.find_all(class_='trend-graph')[2].get_text()
        new_trend = trends.replace('\n','')
        new_trend1 = new_trend.replace('Gem. vierkantemeter prijs','')
        m2prijs,change = new_trend1.split('                ',1)
        change1 = change.replace(' t.o.v. vorige maand            ', '')
        trend_list2.append({'City':city_name1, 'Gem. m2 prijs':m2prijs, '%Δ M2 prijs (t.o.v vorige maand)':change1})
    return(trend_list2)

In [136]:
m2_prijs = extract_city_trends2(page_urls)

We generate a Pandas dataframe of the data we just collected for all municipalities.

In [137]:
pd.DataFrame(m2_prijs)

,City,Gem. m2 prijs,%Δ M2 prijs (t.o.v vorige maand)
0,Alphen-Chaam,€ 2.430,-23.27%
1,Altena,€ 2.928,2.59%
2,Asten,€ 2.809,4.08%
3,Baarle-Nassau,€ 3.733,48.08%
4,Bergeijk,€ 3.427,3.82%
...,...,...,...
56,Vught,€ 3.835,1.64%
57,Waalre,€ 3.734,-11.07%
58,Waalwijk,€ 2.724,-6.17%
59,Woensdrecht,€ 2.719,3.46%


### Percentage overboden (Quadrant 4 Trend data)

In [142]:
def extract_city_trends3(page_urls): #STATUS: change the input still to the correct list name for all municipalities of all provinces!! 
    trend_list3 = []
    for page_url in page_urls:
        driver.get(page_url) #FEEDBACK UPDATE 
        time.sleep(5) #FEEDBACK UPDATE
        soup = BeautifulSoup(res.text, 'html.parser')
        city_name = soup.find_all('h2')[0].get_text()
        city_name1 = city_name.replace('Woningmarkt','')
        trends = soup.find_all(class_='trend-graph')[3].get_text()
        new_trend = trends.replace('\n','')
        new_trend1 = new_trend.replace('Percentage overboden','')
        overboden,change = new_trend1.split('                ',1)
        change1 = change.replace(' t.o.v. vorige maand            ', '')
        trend_list3.append({'City':city_name1, '% Vraagprijs overboden':overboden, '%Δ Overboden (t.o.v vorige maand)': change1})
    return(trend_list3)

In [143]:
perc_overboden = extract_city_trends3(page_urls) #use the function 
print(perc_overboden) #check if it works

We generate a Pandas dataframe of the data we just collected for all municipalities.

In [144]:
pd.DataFrame(perc_overboden)

,City,% Vraagprijs overboden,%Δ Overboden (t.o.v vorige maand)
0,Alphen-Chaam,1.63%,-2.14%
1,Altena,8.79%,-0.22%
2,Asten,7.15%,-0.35%
3,Baarle-Nassau,15.02%,17.89%
4,Bergeijk,3.36%,1.60%
...,...,...,...
56,Vught,6.89%,-0.22%
57,Waalre,10.64%,0.37%
58,Waalwijk,5.48%,1.02%
59,Woensdrecht,7.02%,2.56%


## Step 3b: Gather all data for all municipalties: Besteedbaar inkomen and # inhabitants. 

For each municipality we now extract the Besteedbaar inkomen per huishouden, the number of inhabitants and its % change t.o.v. the year before.

### Besteedbaar inkomen 

In [145]:
def extract_besteedbaar(page_urls): #STATUS: change the input still to the correct list name for all municipalities of all provinces!! 
    besteed_inkomen = []
    for page_url in page_urls:
        driver.get(page_url) #FEEDBACK UPDATE
        time.sleep(5) #FEEDBACK UPDATE
        soup = BeautifulSoup(res.text, 'html.parser')
        city_name = soup.find_all('h2')[0].get_text()
        city_name1 = city_name.replace('Woningmarkt','')
        inkomen = soup.find_all(class_='detail__income huizenzoeker-card single-value-graph-container')[0].get_text()
        new_inkomen = inkomen.replace('\n','')
        new_inkomen1 = new_inkomen.replace('Besteedbaar Inkomen Per Huishouden','')
        besteed_inkomen.append({'City':city_name1, 'Besteedbaar inkomen (per huishouden)':new_inkomen1})
    return(besteed_inkomen)

In [146]:
besteed_inkomen = extract_besteedbaar(page_urls) #use the function 
print(besteed_inkomen) #check whether it works

We generate a Pandas dataframe of the data we just collected for all municipalities

In [148]:
pd.DataFrame(besteed_inkomen)

,City,Besteedbaar inkomen (per huishouden)
0,Alphen-Chaam,€ 45.700
1,Altena,€ 43.400
2,Asten,€ 40.200
3,Baarle-Nassau,€ 37.700
4,Bergeijk,€ 43.600
...,...,...
56,Vught,€ 43.500
57,Waalre,€ 47.300
58,Waalwijk,€ 37.500
59,Woensdrecht,€ 38.800


### Inhabitants

We generate a function extract_inhabitants that will extract the data of the number of inhabitants for all municipalities, and how this number has changed t.o.v. last year.

*Insert code for this still*

## Step 5: Merging all data gathered (by city name) 

For the final output (a CSV file, so tabular data) we would want the output of the scraper to be gathered in one single dictionary.

Here we name the Pandas Dataframes for each quadrant of the trend data we extracted, and also for the besteedbaar inkomen and inhabitants data. We set the index of the dataframes to the name of the municipality.

In [150]:
df1 = pd.DataFrame(gem_vraagprijs2).set_index('City') #gemiddelde vraagprijs

In [151]:
df2 = pd.DataFrame(verk_woningen2).set_index('City') #aantal verkochte woningen 

In [157]:
df3 = pd.DataFrame(m2_prijs2).set_index('City') #vierkantemeter prijs 

In [158]:
df4 = pd.DataFrame(perc_overboden2).set_index('City') #percentage overboden

In [159]:
df5 = pd.DataFrame(besteed_inkomen).set_index('City') #besteedbaar inkomen 

In [ ]:
#add a dataframe for the inhabitants data, once finished. 

In [160]:
data_frames = [df1,df2,df3,df4,df5] #now we combine the dataframes into one single dataframe

The final output: 

In [162]:
df_merged = reduce(lambda left,right: pd.merge(left,right,on=['City'],how='outer'),data_frames)
df_merged 